In [ ]:
import os

import nibabel as nb
from functions import clean_func_image, edit_events_factors_stability, estimate_stability, extract_features_for_stab
from nilearn import image, plotting

In [ ]:
data_root = "/Volumes/T7/BIDS-BRAINPLAYBACK-TASK2"
# data_root = "/DATAPOOL/BRAINPLAYBACK/BIDS-BRAINPLAYBACK-TASK2"  # sim01 dir

derivatives_dir = os.path.join(data_root, "derivatives")
fmriprep_dir = os.path.join(derivatives_dir, "fmriprep23")  # fmriprep output

output_func_dir = os.path.join(derivatives_dir, "func_clean")  # for cleaned bold images
output_mask_stab_dir = os.path.join(derivatives_dir, "mvpa_08_stability_bold", "stab_factors_masks")  # for stability masks
output_feat_stab_dir = os.path.join(derivatives_dir, "mvpa_08_stability_bold", "stab_factors_features")  # for the features

In [ ]:
# check if all these folders exist, otherwise create them
for dir_ in [output_func_dir, output_mask_stab_dir, output_feat_stab_dir]:
    if not os.path.exists(dir_):
        os.makedirs(dir_)
        print(f"Created directory: {dir_}")

In [ ]:
subject_list = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20"]

# Fetch and resample gray matter mask

In [ ]:
from templateflow import api as tflow

t1w_img = tflow.get("MNI152NLin2009cAsym", desc="brain", resolution=2, suffix="T1w", extension="nii.gz")
t1w_gm_img = tflow.get("MNI152NLin2009cAsym", label="GM", suffix="probseg", resolution=2, extension="nii.gz")

In [ ]:
# reference image
example_func_file = os.path.join(
    fmriprep_dir, "sub-02", "ses-01", "func", "sub-02_ses-01_task-02a_run-1_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz"
)

# load, threshold, and resample the GM mask
gm_prob_mask = nb.load(t1w_gm_img)
example_func = nb.load(example_func_file)

gm_mask = image.math_img("img > 0.15", img=gm_prob_mask)
gm_mask_resampled = image.resample_img(gm_mask, target_affine=example_func.affine, target_shape=example_func.shape[0:3], interpolation="nearest")

plotting.view_img(gm_mask_resampled, bg_img=example_func.slicer[..., 0], symmetric_cmap=False, colorbar=True, title="GM mask", opacity=0.5)

## Clean and extract features

In [ ]:
for subject in subject_list:
    # for run in ["1", "3", "4"]: # this one is for sub-18
    for run in ["1", "2", "3", "4"]:
        if subject == "18" and run == "2":
            continue  # skip this run for subject 18
        img_clean = clean_func_image(fmriprep_dir, output_func_dir, gm_mask_resampled, subject, run, overwrite=False)
        events_split = edit_events_factors_stability(data_root, subject, run)
        extract_features_for_stab(img_clean, events_split, output_feat_stab_dir, subject, run)

    estimate_stability(output_feat_stab_dir, output_mask_stab_dir, subject, n_jobs=8)